In [ ]:
# | default_exp routes.user


In [ ]:
# | exporti
import utils.DictDot as dd
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda


# SEARCH and GET Routes


## Search Users


In [ ]:
# | export
async def get_all_users(
    auth: dmda.DomoAuth, debug_api: bool = False
) -> rgd.ResponseGetData:
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    return await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api
    )

#### sample implementation of get_all_users


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_all_users(auth=token_auth)
pd.DataFrame(res.response[0:5])

,id,invitorUserId,displayName,userName,emailAddress,accepted,userType,timeZone,modified,created,role,roleId,rights,active,pending,systemUser,anonymous
0,0,2,monitor,monitor,monitor@domo.com,False,DOMO_SUPPORT,Etc/GMT+7,1588877394091,1464820854,Privileged,1,31.0,True,True,True,True
1,1006847540,1893952720,Marc-Anton Clavel,marcanton.clavel@domo.com,marcanton.clavel@domo.com,False,USER,NaN,1665166064000,1618579073,Privileged,2,31.0,True,True,False,True
2,1012895591,1893952720,JeMiller,JeMiller@marketaxess.com,JeMiller@marketaxess.com,True,USER,NaN,1657051684429,1657049419,NaN,2097317660,NaN,True,False,False,False
3,1024352013,1893952720,Michael Shrifteylik,mshrifteylik@sportsresearch.com,mshrifteylik@sportsresearch.com,True,USER,NaN,1669658030028,1667838056,Privileged,2,31.0,True,False,False,False
4,1026684838,1893952720,Ghofrane Yahmadi,ghofrane.yahmadi@audibene.de,ghofrane.yahmadi@audibene.de,False,USER,NaN,1618579110000,1618579109,Privileged,2,31.0,True,True,False,True


In [ ]:
# | export
def generate_search_users_body_by_id(user_ids: list[str]) -> dict:
    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "filters": [
            {"field": "id", "filterType": "value",
                "values": user_ids, "operator": "EQ"}
        ],
    }


In [ ]:
# | export
def generate_search_users_body_by_email(user_email_ls: list[str]) -> dict:
    """search does not appear to be case sensitive"""

    return {
        # "showCount": true,
        # "count": false,
        "includeDeleted": False,
        "includeSupport": False,
        "limit": 200,
        "offset": 0,
        "sort": {"field": "displayName", "order": "ASC"},
        "filters": [
            {"filterType": "text", "field": "emailAddress", "text": " ".join(user_email_ls)}
        ],
    }


In [ ]:
user_email_ls = ['jae@onyxreporting.com',
                 'marcanton.clavel@domo.com']

generate_search_users_body_by_email(user_email_ls)


{'includeDeleted': False,
 'includeSupport': False,
 'limit': 200,
 'offset': 0,
 'sort': {'field': 'displayName', 'order': 'ASC'},
 'filters': [{'filterType': 'text',
   'field': 'emailAddress',
   'text': 'jae@onyxreporting.com marcanton.clavel@domo.com'}]}

In [ ]:
#| export
def process_v1_search_users_user(obj_user):
    dd_user = dd.DictDot(obj_user)

    return {
        "id": dd_user.id,
        "displayName": dd_user.displayName,
        "roleId": dd_user.roleId,
        "userName": dd_user.userName,
        "emailAddress": dd_user.emailAddress,
    }


def process_v1_search_users(res_user_v1_search: rgd.ResponseGetData):
    obj_users = res_user_v1_search.response.get("users")

    return [process_v1_search_users_user(obj_user) for obj_user in obj_users]

In [ ]:
# | export
async def search_users(
    auth: dmda.DomoAuth,
    body: dict,
    debug_api: bool = False,
    process_users: bool = True,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/search"

    res = await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

    if process_users and res.is_success:
        res.response = process_v1_search_users(res)

    return res

#### sample implementation of search_users


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

search_email_body = generate_search_users_body_by_email(
    user_email_ls= ["jae@onyxreporting.com"]
)

res = await search_users(auth=token_auth, body=search_email_body, process_users=True, debug_api=False)

pd.DataFrame(res.response)

,id,displayName,roleId,userName,emailAddress
0,1893952720,Jae Wilson,1,jae@onyxreporting.com,jae@onyxreporting.com


## Search Publish


In [ ]:
#| export
async def search_virtual_user_by_subscriber_instance(
    auth: dmda.DomoAuth,
    subscriber_instance,
    debug_api: bool = False,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/proxy_user/domain/"

    body = {"domains": [f"{subscriber_instance}.domo.com"]}

    return await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )

# CRUD Routes and Attributes


In [ ]:
# | export
async def create_user(
    auth: dmda.DomoAuth,
    display_name: str,
    email: str,
    role_id: int,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users"

    body = {"displayName": display_name, "detail": {
        "email": email}, "roleId": role_id}

    return await gd.get_data(
        url=url, method="POST", body=body, auth=auth, debug_api=debug_api
    )


In [ ]:
# | export
async def reset_password(
    auth: dmda.DomoAuth,
    user_id: str,
    new_password: str,
    debug_api: bool = False,
) -> rgd.ResponseGetData:

    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/password"

    body = {"domoUserId": user_id, "password": new_password}

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
    )


In [ ]:
# | export
async def request_password_reset(
    domo_instance: str, email: str, 
    locale="en-us", debug_api: bool = False
):
    url = f"https://{domo_instance}.domo.com/api/domoweb/auth/sendReset"

    params = {"email": email, "local": locale}

    return await gd.get_data(
        url=url, method="GET", params=params, auth=None, debug_api=debug_api
    )

In [ ]:
# | export
async def set_user_landing_page(
    auth: dmda.DomoAuth, 
    user_id: str, page_id: 
    str, debug_api: bool = False
):

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/landings/target/DESKTOP/entity/PAGE/id/{page_id}/{user_id}"

    return await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        # body = body,
        debug=debug,
    )

Bad pipe message: %s [b'\xb5,\x18\xc3\\\xaa\xf4\x7f\xd7l\xe1\xd8\xa4\xf1\xc6\xbe<; G\xd6\x7f9\xf1O5\xe5\xd7_\xc9\xec\xd9\x906\x01_U}.\xf1\x19\x00`']
Bad pipe message: %s [b'\xc1\xc8\x83p\xc4\xaa\xe0}\xc6Fl\xddb2\xdaS5\x82\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0']
Bad pipe message: %s [b"\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0"]
Bad pipe message: %s [b'3\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0']
Bad pipe message: %s [b'=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00']
Bad pipe message: %s [b'\xc4W\xa4B\x9d8\xc12\xe6\xc2[/u\xfd\xe4u\x1a7\x00\x00\xa6\xc0,\xc00\x00\xa3\x00\x9f\